In [1]:
from pyspark.sql import SparkSession
# May take a little while on a local computer
spark = SparkSession.builder.appName("missingdata").getOrCreate()

In [2]:
df = spark.read.csv("ContainsNull.csv",header=True,inferSchema=True)

In [3]:
df.show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp2|     | null|
|emp3|     |345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



### drop na data

In [4]:
# Drop any row that contains missing data
df.na.drop().show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp3|     |345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [5]:
# Has to have at least 2 NON-null values
df.na.drop(thresh=2).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp2|     | null|
|emp3|     |345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [6]:
df.na.drop(subset=["Sales"]).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp3|     |345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [7]:
df.na.drop(how='any').show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp3|     |345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [8]:
df.na.drop(how='all').show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp2|     | null|
|emp3|     |345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



### 填充空缺值

In [9]:
df.na.fill('NEW VALUE').show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp2|     | null|
|emp3|     |345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [10]:
df.na.fill(0).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John|  0.0|
|emp2|     |  0.0|
|emp3|     |345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [11]:
df.na.fill('No Name',subset=['Name']).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp2|     | null|
|emp3|     |345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [12]:
from pyspark.sql.functions import mean
mean_val = df.select(mean(df['Sales'])).collect()

# Weird nested formatting of Row object!
mean_val[0][0]

400.5

In [13]:
mean_sales = mean_val[0][0]

In [14]:
df.na.fill(mean_sales,["Sales"]).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John|400.5|
|emp2|     |400.5|
|emp3|     |345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [15]:
# One (very ugly) one-liner
df.na.fill(df.select(mean(df['Sales'])).collect()[0][0],['Sales']).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John|400.5|
|emp2|     |400.5|
|emp3|     |345.0|
|emp4|Cindy|456.0|
+----+-----+-----+

